## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embeddings, load one from gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [28]:
# download the data:
!wget https://yadi.sk/i/BPQrUu1NaTduEw
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2025-04-01 22:00:08--  https://yadi.sk/i/BPQrUu1NaTduEw
Resolving yadi.sk (yadi.sk)... 87.250.250.50, 2a02:6b8::2:50
Connecting to yadi.sk (yadi.sk)|87.250.250.50|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://disk.yandex.ru/i/BPQrUu1NaTduEw [following]
--2025-04-01 22:00:09--  https://disk.yandex.ru/i/BPQrUu1NaTduEw
Resolving disk.yandex.ru (disk.yandex.ru)... 87.250.250.50, 2a02:6b8::2:50
Connecting to disk.yandex.ru (disk.yandex.ru)|87.250.250.50|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34811 (34K) [text/html]
Saving to: ‘BPQrUu1NaTduEw’

BPQrUu1NaTduEw      100%[===================>]  34.00K  71.9KB/s    in 0.5s    

2025-04-01 22:00:11 (71.9 KB/s) - ‘BPQrUu1NaTduEw’ saved [34811/34811]



In [29]:
import numpy as np

with open("/kaggle/input/quora-txt/quora.txt", encoding="utf-8") as file:
    data = list(file)

data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ a typical first step for an NLP task is to split raw data into words.
The text we're working with is in raw format: with all the punctuation and smiles attached to some words, so a simple str.split won't do.

Let's use __`nltk`__ - a library that handles many NLP tasks like tokenization, stemming or part-of-speech tagging.

In [24]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()


In [3]:
# TASK: lowercase everything and extract tokens with tokenizer. 
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(x.lower()) for x in data]

In [4]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [5]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ as the saying goes, there's more than one way to train word embeddings. There's Word2Vec and GloVe with different objective functions. Then there's fasttext that uses character-level models to train word embeddings. 

The choice is huge, so let's start someplace small: __gensim__ is another nlp library that features many vector-based models incuding word2vec.

In [6]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, 
                 vector_size=32,      # embedding vector size
                 min_count=5,  # consider words that occured at least 5 times
                 window=5).wv  # define context as a 5-word window around the target word

# From gensim docs
# vw: This object essentially contains the mapping between words and embeddings.
# After training, it can be used directly to query those embeddings in various ways.

In [24]:
# now you can get word vectors !
model.get_vector('anything')

array([-2.6697218e+00, -1.0987531e+00, -8.8427648e-02,  4.6548023e+00,
        1.5939209e+00,  1.7997106e+00,  9.7294503e-01, -2.8349853e+00,
        2.6793605e-01,  3.1394331e+00, -6.0016018e-01,  2.1487944e+00,
        4.6371236e+00,  1.6483411e+00,  2.5478921e+00, -8.0527134e-02,
       -1.7196650e-04, -8.5104096e-01,  1.2144747e+00, -3.1435320e+00,
       -3.5916178e+00, -2.7394590e-01, -7.7842081e-01, -1.1290724e+00,
        9.9292254e-01, -2.2845592e+00,  2.1012273e-01, -1.0542731e+00,
       -1.2861566e-01,  1.2078704e+00, -1.2417511e+00,  5.3815019e-01],
      dtype=float32)

In [25]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9402197003364563),
 ('cheese', 0.9141132831573486),
 ('butter', 0.9135099053382874),
 ('fruit', 0.9130918383598328),
 ('beans', 0.9085392951965332),
 ('sauce', 0.9069536924362183),
 ('beer', 0.9007349610328674),
 ('potato', 0.8993415832519531),
 ('flour', 0.8988161087036133),
 ('chocolate', 0.8984243273735046)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

After being downloaded for the first time (or if you manually delete it), the model is saved in the `~/gensim_data` or `%USER_PATH%/gensim_data` directory. This can be checked seting the return_path parameter to True.

In [1]:
import gensim.downloader as api
api.info()

{'corpora': {'semeval-2016-2017-task3-subtaskBC': {'num_records': -1,
   'record_format': 'dict',
   'file_size': 6344358,
   'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py',
   'license': 'All files released for the task are free for general research use',
   'fields': {'2016-train': ['...'],
    '2016-dev': ['...'],
    '2017-test': ['...'],
    '2016-test': ['...']},
   'description': 'SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collected data is given in sections 3 and 4.1 of the task paper http://alt.qcri.org/semeval2016/task3/data/uploads/semeval2016-task3-report.pdf linked in section “Papers” of https://github.com/RaRe-Technologies/gensim-data/issues/18.',
   'checksum': '701ea67acd82e75f95e1d8e62fb0ad29',
   'file_name': 'se

In [2]:
model = api.load('glove-twitter-100')

[=============================================-----] 90.8% 351.6/387.1MB downloaded

In [3]:
model.most_similar(positive=["coder", "money"], negative=["brain"])

[('broker', 0.5820155143737793),
 ('bonuses', 0.5424473285675049),
 ('banker', 0.5385112762451172),
 ('designer', 0.5197197794914246),
 ('merchandising', 0.4964233338832855),
 ('treet', 0.49220192432403564),
 ('shopper', 0.4920562207698822),
 ('part-time', 0.49128273129463196),
 ('freelance', 0.4843311905860901),
 ('aupair', 0.4796452820301056)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [4]:
words = model.index_to_key[:1000] 

words[::100]

['<user>',
 '_',
 'please',
 'apa',
 'justin',
 'text',
 'hari',
 'playing',
 'once',
 'sei']

In [5]:
# for each word, compute it's vector with model
import numpy as np 
word_vectors = model[words]

In [6]:
assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

The simplest linear dimensionality reduction method is **P**rincipial **C**omponent **A**nalysis.

In geometric terms, PCA tries to find axes along which most of the variance occurs. The "natural" axes, if you wish.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Under the hood, it attempts to decompose object-feature matrix $X$ into two smaller matrices: $W$ and $\hat W$ minimizing _mean squared error_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [14]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# map word vectors onto 2d plane with PCA. Use good old sklearn api (fit, transform)
# after that, normalize vectors to make sure they have zero mean and unit variance
word_vectors_pca = PCA(n_components = 2)
word_vectors_pca = word_vectors_pca.fit_transform(word_vectors)
scaler = StandardScaler()
normalized_word_vectors_pca = scaler.fit_transform(word_vectors_pca) 
# and maybe MORE OF YOUR CODE here :)

In [18]:
assert normalized_word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(normalized_word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - normalized_word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [11]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [12]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

In [17]:
draw_vectors(normalized_word_vectors_pca[:, 0], normalized_word_vectors_pca[:, 1], token=words)

figure(id='p1055', ...)

### Visualizing neighbors with t-SNE
PCA is nice but it's strictly linear and thus only able to capture coarse high-level structure of the data.

If we instead want to focus on keeping neighboring points near, we could use TSNE, which is itself an embedding method. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [20]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: don't panic it may take a minute or two to fit.
# normalize them as just lke with pca


word_tsne = TSNE(n_components = 2)
word_tsne = word_tsne.fit_transform(word_vectors)
scaler = StandardScaler()
word_tsne = scaler.fit_transform(word_tsne)



In [21]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1106', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [31]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating its word embeddings.
    - Lowercases the phrase
    - Tokenizes the phrase
    - Averages word vectors for known tokens
    - Returns zero vector if all tokens are unknown
    """
    tokens = tokenizer.tokenize(phrase.lower())
    vectors = [model[token] for token in tokens if token in model]
    
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size, dtype=np.float32)


In [32]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))
assert np.array_equal(get_phrase_embedding("thisisgibberish"), np.zeros([model.vector_size], dtype='float32')), "corner case for all missing words should be handled as described in the function comments"

In [36]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase) for phrase in chosen_phrases])

In [38]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [39]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [40]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

figure(id='p1157', ...)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [48]:
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [49]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    query = get_phrase_embedding(query)
    for vector in data_vectors:
        cosine = vector @ query / np.linalg.norm(vector) / np.linalg.norm(query)
        print(cosine)
    
    

In [ ]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize `find_nearest` with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.